# GPT Modelnetv2 Testing

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense

In [ ]:
# TODO: add 

In [6]:
tfds.list_builders()


['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 'conll2003',
 'controlled_noisy_web_labels'

In [8]:
dataset = tfds.load('voc', split=['train', 'test', 'val'], shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  50%|█████     | 1/2 [06:38<06:38, 398.70s/ url]
Extraction completed...: 0 file [06:38, ? file/s]
Dl Completed...:  50%|█████     | 1/2 [06:38<06:38, 398.72s/ url]


NonMatchingChecksumError: Artifact http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar, downloaded to /Users/davidperez/tensorflow_datasets/downloads/pjreddi.com_media_files_VOCtest_6-Nov-2007-uyfHB4aCy4ZRNbxEOl0zsGlfk4orDpuRRs8mcmQyGk.tar.tmp.ba7dac9b029c4ea6867c833ee5007cc8/VOCtest_06-Nov-2007.tar, has wrong checksum:
* Expected: UrlInfo(size=430.13 MiB, checksum='6836888e2e01dca84577a849d339fa4f73e1e4f135d312430c4856b5609b4892', filename='VOCtest_06-Nov-2007.tar')
* Got: UrlInfo(size=17.81 MiB, checksum='87cad5e2ebb4b7923af6ec974ad9cc8659f65685693b8f860a35f4a146e919bd', filename='VOCtest_06-Nov-2007.tar')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror

In [ ]:
class CustomMobileNetV2(tf.keras.Model):
    def __init__(self, num_classes=1000, alpha=1.0):
        super(CustomMobileNetV2, self).__init__()

        self.alpha = alpha

        # Define the MobileNetV2 architecture
        self.conv1 = Conv2D(filters=int(32 * self.alpha), kernel_size=(3, 3), strides=(2, 2), padding='same')
        self.bn1 = BatchNormalization()
        self.relu1 = ReLU()

        self.block1 = self._inverted_residual_block(int(16 * self.alpha), 1, 1)
        self.block2 = self._inverted_residual_block(int(24 * self.alpha), 2, 6)
        self.block3 = self._inverted_residual_block(int(32 * self.alpha), 3, 6)
        self.block4 = self._inverted_residual_block(int(64 * self.alpha), 4, 6)
        self.block5 = self._inverted_residual_block(int(96 * self.alpha), 3, 6)
        self.block6 = self._inverted_residual_block(int(160 * self.alpha), 3, 6)
        self.block7 = self._inverted_residual_block(int(320 * self.alpha), 1, 6)

        self.conv2 = Conv2D(filters=int(1280 * self.alpha), kernel_size=(1, 1), strides=(1, 1), padding='same')
        self.bn2 = BatchNormalization()
        self.relu2 = ReLU()

        self.avgpool = GlobalAveragePooling2D()
        self.fc = Dense(num_classes, activation='softmax')

    def _inverted_residual_block(self, filters, expansion, strides):
        out_channels = int(filters * self.alpha)

        return tf.keras.Sequential([
            Conv2D(filters=expansion * out_channels, kernel_size=(1, 1), padding='same'),
            BatchNormalization(),
            ReLU(),
            DepthwiseConv2D(kernel_size=(3, 3), strides=strides, padding='same'),
            BatchNormalization(),
            ReLU(),
            Conv2D(filters=out_channels, kernel_size=(1, 1), padding='same'),
            BatchNormalization()
        ])

    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.avgpool(x)
        x = self.fc(x)

        return x


# You can specify the number of classes and alpha (width multiplier) as desired
num_classes = 1000  # Adjust this based on your classification problem
alpha = 1.0  # Adjust this to control the width of the network

model = CustomMobileNetV2(num_classes=num_classes, alpha=alpha)


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with your data (X_train and y_train are placeholders, use your own data)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
